In [1]:
from bs4 import BeautifulSoup
import requests
import json
import pandas as pd
import time
import datetime as dt


In [224]:
start_url = f'http://www.kicktraq.com/search/?page=1&find=in:"technology"&sort=new'

In [26]:
req = requests.get(start_url)
soup = BeautifulSoup(req.content)

In [42]:
project_list = soup.find('div', {'id': 'project-list'})
projects = project_list.find('div', {'class': 'projects'}).find_all('div', {'class': 'project'})
next_url = project_list.find('div', {'class': 'paging'}).find('a', {'class': 'prn'})['href']
next_url = ''.join([f'http://www.kicktraq.com/search/', next_url, f'&sort=new'])

http://www.kicktraq.com/search/?page=2&find=in:"technology"&sort=new


In [43]:
print(len(projects))

15


In [90]:
proj = projects[0]
info = proj.find('div', {'class': 'project-infobox'})
a = info.find('h2').find()
link = f'http://www.kickstarter.com' + a['href']
name = a.text
info = proj.find('div', {'class': 'project-infobits'})
pledgilizer = info.find('div', {'class': 'project-pledgilizer'})

status = pledgilizer.find('div', {'class': 'project-pledgilizer-top'}).find().text.lower()
if status == 'closed':
    funded = pledgilizer.find('div', {'class': 'project-pledgilizer-mid'}).find().text[:-1]
    if int(funded) >= 100:
        status = 'success'
    else:
        status = 'failed'
    
funding_info = info.find('div', {'class':'project-details'}).text.split('\n\t\t\t\t\t')
moneys = funding_info[2].split(' ')
goal = moneys[3][1:].replace(',', '')
pledged = moneys[1][1:].replace(',', '')
backers = funding_info[1].split(' ')[1]

'0'

In [114]:
s = f'https://www.kickstarter.com/projects/601285608/scout-the-autonomous-transatlantic-boat/?ref=kicktraq'
f = f'http://www.kickstarter.com/projects/aromaoftacoma/pocket-web-server-kit/?ref=kicktraq'
kick_req = requests.get(f)
kick_soup = BeautifulSoup(kick_req.content)

In [132]:
comments = kick_soup.find('a', {'id': 'comments-emoji'}).find().text

tiers = kick_soup.find('div', {'class': 'NS_projects__rewards_list'}).find('ol').find_all('li')
tier_count = len(tiers)
tier_min = tiers[0]
tier_max = tiers[-1]
tier_cost_min = tiers[0].find('div', {'class': 'pledge__info'}).find('span', {'class': 'money'}).text.split(' ')[1].replace(',', '')
tier_cost_max = tiers[-1].find('div', {'class': 'pledge__info'}).find('span', {'class': 'money'}).text.split(' ')[1].replace(',', '')
tier_min = int(tier_cost_min)
tier_max = int(tier_cost_max)

In [137]:
tiers

[]

In [ ]:
df = pd.DataFrame(columns=['Id', 'Name', 'Url', 'Goal', 'Pledged', 'Backers', 'Tiers', 'TierMin', 'TierMax', 'Status'])
next_url = start_url
pid = 0

while next_url is not None:
    url = next_url
    req = requests.get(url)
    while not req.ok:
        print('Request to ', url, ' failed')
        time.sleep(1)
        req = requests.get(url)
    print('Requested URL: ', url)
    
    soup = BeautifulSoup(req.content)
    project_list = soup.find('div', {'id': 'project-list'})
    projects = project_list.find('div', {'class': 'projects'}).find_all('div', {'class': 'project'})
    next_url = project_list.find('div', {'class': 'paging'}).find('a', {'class': 'prn'}, text='Next >')

    if next_url is not None:
        next_url = ''.join([f'http://www.kicktraq.com/search/', next_url['href'], f'&sort=new'])

    for proj in projects:
        info = proj.find('div', {'class': 'project-infobox'})
        a = info.find('h2').find()
        link = f'http://www.kickstarter.com' + a['href']
        name = a.text
        info = proj.find('div', {'class': 'project-infobits'})
        pledgilizer = info.find('div', {'class': 'project-pledgilizer'})

        status = pledgilizer.find('div', {'class': 'project-pledgilizer-top'}).find().text.lower()
        if status != 'closed':
            continue
        else:
            funded = pledgilizer.find('div', {'class': 'project-pledgilizer-mid'}).find().text[:-1]
            if int(funded) >= 100:
                status = 'success'
            else:
                status = 'failed'

        funding_info = info.find('div', {'class':'project-details'}).text.split('\n\t\t\t\t\t')
        moneys = funding_info[2].split(' ')
        goal = moneys[3][1:].replace(',', '')
        pledged = moneys[1][1:].replace(',', '')
        backers = funding_info[1].split(' ')[1]

        kick_req = requests.get(link)
        while not kick_req.ok:
            print('Request to Kickstarter ', link, ' failed')
            time.sleep(1)
            kick_req = requests.get(link)
            
        kick_soup = BeautifulSoup(kick_req.content)
        comments = kick_soup.find('a', {'id': 'comments-emoji'})
        if comments is None:
            continue
        comments = comments.find().text

        tiers = kick_soup.find('div', {'class': 'NS_projects__rewards_list'}).find('ol').find_all('li', recursive=False)
        tier_count = len(tiers)
        tier_min = 0
        tier_max = 0
        if tier_count > 0:          
            tier_cost_min = tiers[0].find('div', {'class': 'pledge__info'}).find('span', {'class': 'money'}).text.split(' ')[-1].replace(',', '')
            tier_cost_max = tiers[-1].find('div', {'class': 'pledge__info'}).find('span', {'class': 'money'}).text.split(' ')[-1].replace(',', '')
        
        while not tier_cost_min.isnumeric():
            tier_cost_min = '0' if len(tier_cost_min) == 0 else tier_cost_min[1:]
        while not tier_cost_max.isnumeric():
            tier_cost_max = '0' if len(tier_cost_max) == 0 else tier_cost_max[1:]
        tier_min = int(tier_cost_min)
        tier_max = int(tier_cost_max)
        new_row = {'Id': pid, 'Name': name, 'Url': link, 'Goal': goal, 'Pledged': pledged, 'Backers': backers, 'Tiers': tier_count, 'TierMin': tier_min, 'TierMax': tier_max, 'Status': status}
        df = df.append(new_row, ignore_index=True)
        pid += 1
        if pid % 200 == 0:
            df.to_csv('data.csv', index=None)
            print('Backed up ', pid, ' indices')

Requested URL:  http://www.kicktraq.com/search/?page=1&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=2&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=3&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=4&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=5&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=6&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=7&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=8&find=in:"technology"&sort=new
Requested URL:  http://www.kicktraq.com/search/?page=9&find=in:"technology"&sort=new


In [227]:
kick_soup.find('a', {'id': 'comments-emoji'})

In [228]:
kick_soup.find(id='comments-emoji')

In [229]:
link

'http://www.kickstarter.com/projects/ff-hta/hit-them-all/'